In [1]:
library(RColorBrewer)
library(ggpubr)
library(RColorBrewer)
library(ggplot2)
library(ggpubr)

Loading required package: AnVIL

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MultiAssayExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    

### Differences in risk scores

In [4]:
xxx = fread("TCGA.COAD.sampleMap_COAD_clinicalMatrix")
xxx$MSI_ = ifelse(xxx$MSI =="Indeterminate"|xxx$MSI =="",NA,xxx$MSI)
xxx$MSI  = ifelse( xx$MSI_updated_Oct62011 !="",xx$MSI_updated_Oct62011,xxx$CDE_ID_3226963)
xxx$MSI_ = ifelse(xxx$MSI =="Indeterminate"|xxx$MSI =="",NA,xxx$MSI)
xxx = na.omit(xxx)
save(coad_msi,file = "coad_msi.rdata")
load("msi_risk.rdata")
x = as.data.frame(msi_risk[order(msi_risk$riskScore,decreasing = F),])
x = x[-(grep("MSS",head(x,50)[,2])),]

mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
my_comparisons <- list( c("MSI-H", "MSI-L"), c("MSI-H", "MSS"), c("MSI-L", "MSS") ) 
g3 = ggplot(x,aes(MSI,riskScore,fill = MSI)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "riskScore") +
  theme(legend.position = "top") + 
  scale_fill_manual(values = mypalette(22)[c(6,1,2)])+ 
  stat_compare_means(comparisons = my_comparisons)+   # Add global p-value
theme(text = element_text(size = 20))
g3
ggsave("MSI_3.pdf",g3,width = 5,height =5)

### Appling KM analysis in group of MSI

In [ ]:
load("../../Model_lasso_cox/模型构建/芯片建模_TCGA验证/OKModel_risk.rdata")
TCGA_risk_ = TCGA_risk[,1:10]
TCGA_risk_$ID = substr(TCGA_risk_$ID,1,15)
x__ = merge(x,TCGA_risk_,by.x = "ID",by.y ="ID")
x__$MSI_risk = ifelse(x__$MSI == "MSS" & x__$risk == "high","MSS+High risk",
                     ifelse(x__$MSI == "MSS" & x__$risk == "low","MSS+Low risk",
                           ifelse(x__$MSI == "MSI-H" & x__$risk == "low","MSI-H+Low risk",
                                 ifelse(x__$MSI == "MSI-H"& x__$risk == "high","MSI-H+High risk",
                                       ifelse(x__$MSI == "MSI-L"& x__$risk == "high","MSI-L+High risk","MSI-L+Low risk")))))
library(survival)
library(survminer)
fit <- survfit(Surv(OS_time, Status) ~x__[,"MSI_risk"] , data = x__)
g = ggsurvplot(fit,
          pval = TRUE, #conf.int = TRUE,
          #risk.table = TRUE, # 添加风险表
          risk.table.col = "strata", # 根据分层更改风险表颜色
          #linetype = "strata", # 根据分层更改线型
          surv.median.line = "hv", # 同时显示垂直和水平参考线
          #ggtheme = theme_bw(), # 更改ggplot2的主题
          palette =mypalette(22)[c(1:6)],
               legend = c(0.9,0.8),
               legend.title = "",
          legend.labs = names(table(x__[,paste0("MSI_risk")])),ggtheme = theme_minimal(base_size = 15))
g
pdf("MSI_KM.pdf",width = 5,height = 5)
g
dev.off()


### Percentage stacking chart of MSI

In [88]:
MSI_KM = x__
MSI_boxplot = x
save(MSI_KM,MSI_boxplot,file = "MSI_plot.rdata")
ggplot(myresults_pct,aes(x=risk,y=pct,fill=MSI)) +theme_bw() + 
geom_col(width = 0.6)+
scale_y_continuous(label = scales::percent)+scale_fill_manual(values = mypalette(22)[c(6,1,2)])+t
heme( text = element_text(size = 15)))
library(cBioPortalData)
cbio <- cBioPortal()
studies = getStudies(cbio)
head(studies$studyId)
id = "ucec_tcga_pan_can_atlas_2018"
clinical = clinicalData(cbio, id)
colnames(clinical)

### TMB analysis

In [ ]:
library(TCGAbiolinks)
library(tidyverse)
library(maftools)
library(data.table)
query <- GDCquery(
    project = "TCGA-READ", 
    data.category = "Simple Nucleotide Variation",
    data.type = "Masked Somatic Mutation",
    access = "open"
)
GDCdownload(query)
GDCprepare(query, save = T,save.filename = "TCGA-READ_SNP.Rdata")
load(file = "TCGA-COAD_SNP.Rdata")
maf.read<- data
x = fread("mc3.v0.2.8.PUBLIC.maf.gz");x[1:4,1:4]
maf <- read.maf(maf.read)
x = tmb(maf = maf);head(x)
tmb = data.frame(ID =gsub("-","." ,substr(x$Tumor_Sample_Barcode,1,16)),TMB = x$total_perMB,TMBlog = x$total_perMB_log)
load("TCGA_risk.rdata")
risk_tmb = merge(tmb,risk,by= "ID")
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
my_comparisons = list(c("high","low"))
g3 = ggplot(x,aes(risk,TMBlog,fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "log(TMB)") +
  theme(legend.position = "top") + 
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
    stat_compare_means(comparisons = my_comparisons)+# 调整其他文本的字体大小
  theme( text = element_text(size = 20))
#stat_compare_means(aes(group = risk,label = ..p.signif..),method = "wilcox.test",label.x = 1.5,na.rm = T,)
ggsave("TBM_TCGA.pdf",g3,width = 5,height =5)

### KM analysis in TMB group

In [68]:
library(survival)
library(survminer)
load("../../Model_lasso_cox/模型构建/芯片建模_TCGA验证/OKModel_risk.rdata")
TCGA_risk_ = TCGA_risk[,1:10]
x__ = merge(x,TCGA_risk_,by.x = "ID",by.y ="ID_")
x__$TMB_group_risk = ifelse(x__$TMBlog > median(x__$TMBlog)&x__$risk == "high","High TMB+High risk",
                            ifelse(x__$TMBlog > median(x__$TMBlog)&x__$risk == "low","High TMB+Low risk",
                                  ifelse(x__$TMBlog < median(x__$TMBlog)&x__$risk == "low","Low TMB+Low risk","Low TMB+High risk")))
x___ = x__[x__$OS_time<5,]
fit <- survfit(Surv(OS_time, Status) ~x__[,"TMB_group_risk"] , data = x__)
g = ggsurvplot(fit,
          pval = TRUE, #conf.int = TRUE,
          #risk.table = TRUE, # 添加风险表
          risk.table.col = "strata", # 根据分层更改风险表颜色
          #linetype = "strata", # 根据分层更改线型
          surv.median.line = "hv", # 同时显示垂直和水平参考线
          #ggtheme = theme_bw(), # 更改ggplot2的主题
          palette =mypalette(22)[c(1:4)],
               legend = c(0.7,0.8),
               legend.title = "",
          legend.labs = names(table(x__[,paste0("TMB_group_risk")])),ggtheme = theme_minimal(base_size = 15))
pdf("TMB_KM.pdf",width = 5,height = 5)
g
dev.off()
save(tmb,risk_tmb,x,km_tmb,file = "TMB_coad.rdata")

### estimate

In [ ]:
library(estimate)
library(tidyverse)
library(RColorBrewer)
library(ggpubr)
load("../estimate/TCGA/estimate.rdata")
load("../../Model_lasso_cox/模型构建/芯片建模_TCGA验证/OKModel_risk.rdata")
Expr = data.frame(entire_risk[,-c(17662:17670)]%>%t())
Expr = data.frame(TCGA_exp_pd[,-c(1:10)]%>%t())
write.table(Expr,file = "TCGA_exp.txt",row.names = T,quote = F,sep = "\t")
filterCommonGenes(input.f = "TCGA_exp.txt", 
                  output.f = "TCGA_exp.gct", 
                  id = "GeneSymbol")
estimateScore(input.ds = "TCGA_exp.gct",
              output.ds = "TCGA_score_T.gct", 
              platform = "affymetrix")#illumina;affymetrix
scores <- read.table("TCGA_score_T.gct",skip = 2,header = T)
rownames(scores) <- scores[,1]
scores <- t(scores[,3:ncol(scores)])
scores
TCGA_estimate = scores
save(entire_estimate,TCGA_estimate,file = "estimate.rdata")


### ssgsea

In [ ]:
library(GSVA)
library(tidyverse)
library(data.table)
library(tidyr)
library(pheatmap)
library(RColorBrewer)
library(ggpubr)
load("Chip2_pd_exp for lasso_cox.rdata")
load("OKModel_risk.rdata")
entire_ = entire[,-c(17662:17668)]%>%t()
TCGA_exp_ = TCGA_exp_pd[,-c(1:10)]%>%t()
read.csv("29.csv") %>%gather(set,gene)%>%unique()->x
x = x[x$gene != "",]
geneset29 = split(x$gene,x$set)
ssGSEA_matrix <- gsva(expr = as.matrix(TCGA_exp_), 
                      gset.idx.list = geneset29,
                      method = 'ssgsea',kcdf = 'Gaussian',abs.ranking = TRUE)
scale_gsva_matrix <- t(scale(t(ssGSEA_matrix)))
scale_gsva_matrix[scale_gsva_matrix< -2] <- -2
scale_gsva_matrix[scale_gsva_matrix>2] <- 2


### GSVA

In [ ]:
library(tidyverse)
library(clusterProfiler)
library(msigdbr)  
library(GSVA) 
library(GSEABase)
library(pheatmap)
library(limma)
library(BiocParallel)

In [3]:
KEGG_df_all <-  msigdbr(species = "Homo sapiens", # Homo sapiens or Mus musculus
                        category = "C2",
                        subcategory = "CP:KEGG") 
KEGG_df <- dplyr::select(KEGG_df_all,gs_name,gs_exact_source,gene_symbol)
kegg_list <- split(KEGG_df$gene_symbol, KEGG_df$gs_name) 


GO_df_all <- msigdbr(species = "Homo sapiens",
                     category = "C5")  
GO_df <- dplyr::select(GO_df_all, gs_name, gene_symbol, gs_exact_source, gs_subcat)
GO_df <- GO_df[GO_df$gs_subcat!="HPO",]
go_list <- split(GO_df$gene_symbol, GO_df$gs_name)

Hallmark <-  msigdbr(species = "Homo sapiens", # Homo sapiens or Mus musculus
                        category = "H") 
Hallmark_df <- dplyr::select(Hallmark,gs_name,gs_exact_source,gene_symbol)
Hallmarklist <- split(Hallmark_df$gene_symbol, Hallmark_df$gs_name) ##按照gs_name给gene_symbol分组

load("OKModel_risk.rdata")
dat = as.matrix(t(entire_risk[,1:17661]))
geneset <- Hallmarklist#（kegg_list，go_list）
gsva_mat <- gsva(expr=dat, 
               gset.idx.list=geneset, 
               kcdf="Gaussian" ,#"Gaussian" for logCPM,logRPKM,logTPM, "Poisson" for counts
               verbose=T, 
               parallel.sz = parallel::detectCores())


### heatmap

In [16]:
gsva_mat = read.csv("gsva_go_matrix.csv",header = T, row.names = 1)
exp="high"
ctr="low"
design <- model.matrix(~0+factor(group_list$group))
colnames(design) <- levels(factor(group_list$group))
rownames(design) <- colnames(gsva_mat)
contrast.matrix <- makeContrasts(contrasts=paste0(exp,'-',ctr),  #"exp/ctrl"
                                 levels = design)
fit1 <- lmFit(gsva_mat,design)                 
fit2 <- contrasts.fit(fit1, contrast.matrix) 
efit <- eBayes(fit2)                         
summary(decideTests(efit,lfc=0, p.value=0.05)) 
tempOutput <- topTable(efit, coef=paste0(exp,'-',ctr), n=Inf)
degs <- na.omit(tempOutput) 
write.csv(degs,"gsva_DDR_degs.results.csv")
degs =  read.csv("gsva_go_degs.results.csv", header = T, row.names = 1)
UP = rownames(degs[order(degs$logFC,decreasing = T),])[1:10];DOWN = rownames(degs[order(degs$logFC,decreasing = F),])[1:10]
keep = c(UP,DOWN)
padj_cutoff=0.05
log2FC_cutoff=log2(1)
keep <- rownames(degs[degs$adj.P.Val < padj_cutoff & abs(degs$logFC)>log2FC_cutoff, ])
length(keep)
keep = c(UP,DOWN)
dat <- gsva_mat[keep,]
dat = dat[,order(group_list$group)]
annotation_row = data.frame(row.names = rownames(dat),anno = substr(rownames(dat),1,30))
annotation_row$... = "..."
annotation_row$anno = str_c(annotation_row$anno ,annotation_row$...)
annotation_row$anno = gsub("_"," ",annotation_row$anno)
groupcolor = c("#9d4351","#6d5d80");names(groupcolor) = c("high","low")
ann_colors = list(group = groupcolor)
pheatmap::pheatmap(dat, 
                   fontsize_row = 8,
                   cluster_rows = F,
                   cluster_cols = F,
                   height = 5,
                   width=7,
                   scale = "row",
                   annotation_col = group_list,
                   annotation_colors = ann_colors,
                   show_colnames = F,
                   show_rownames = T,
                   labels_row  = annotation_row$anno,
                   color=colorRampPalette(c("navy","white","firebrick3"))(100),
                  # filename = paste0('GSVA_go_heatmap.pdf')
                   filename = paste0('GSVA_kegg_heatmap.pdf')
                  )


In [121]:
### DDR related pathway in hallmarks

In [458]:
DDR_set = read.table("clipboard",header = T,sep = "\t")
DDR_set$Pathway = gsub("\\s*\\(.*","",DDR_set$Pathway)
DDR_set$Genes = gsub("\\s*","",DDR_set$Genes)
DDR_set$Genes = strsplit(DDR_set$Genes,",")
DDR_df = data.frame()
for(i in 1:8){
    
df= data.frame(set  = DDR_set[i,1],genes = unlist(DDR_set[i,2]))
DDR_df = rbind(DDR_df,df)    
}
DDR_list = split(DDR_df$genes,DDR_df$set)
DDR_list[["NER_"]] = DDR_list[["NER"]][2:length(DDR_list[["NER"]])]
DDR_list_ = DDR_list[6]
geneset <- DDR_list

gsva_mat <- gsva(expr=dat, 
               gset.idx.list=geneset, 
               kcdf="Gaussian" ,#"Gaussian" for logCPM,logRPKM,logTPM, "Poisson" for counts
               verbose=T, 
               parallel.sz = parallel::detectCores())#调用所有核
library(RColorBrewer)
library(tidyr)
library(ggpubr)
data = data.frame(t(gsva_mat))
data$ID = rownames(data);
group_list$ID = rownames(group_list)
data_ = merge(data,group_list,by = "ID")
data_ = gather(data_,sets,values, -c(group,ID))
colnames(data_)[2] = "risk"
my_comparisons = list(c("high","low"))
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
g3 = ggplot(data_,aes(sets,values,fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "Gene expression") +
  theme(legend.position = "top") + 
  #theme(axis.text.x = element_text(angle=90,vjust = 0.7, hjust = 1))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
stat_compare_means(aes(group = risk,label = ..p.signif..),method = "wilcox.test")+
theme(text =element_text(size = 20))

### correlation analysis among score and DDR pathway

In [ ]:
library(ggExtra)
theme_global <- theme_bw()+theme(panel.grid.major = element_blank(),panel.grid.minor=element_blank()) + theme(aspect.ratio = 1)+  theme(
    text = element_text(size = 15),  # 调整所有文本的字体大小
    axis.text = element_text(size = 15),  # 调整坐标轴标签的字体大小
    axis.title = element_text(size = 15),  # 调整坐标轴标题的字体大小
    legend.title = element_text(size = 14),  # 调整图例标题的字体大小
    legend.text = element_text(size = 12)  # 调整图例文本的字体大小
  )
risk_colors <- c("low" = "#469f6f", "high" = "#e41a1c") 
data = read.csv("gsva_Hallmark_matrix.csv",header = T,row.names = 1)
data = as.data.frame(t(data))
data_ = data[,c(12,35,14,18)]
data_$ID = rownames(data_)
df = data.frame(ID = rownames(entire_risk),riskScore = entire_risk[,17669],risk =entire_risk[,17670])
data__ =merge(df,data_, by = "ID")

colnames(data__)[4:7] = gsub("_"," ",substring(colnames(data__)[4:7],10))
colnames(data__)[6] = "EMT"
pp = list()
for(i in 4:ncol(data__)){
g1 = ggscatter(data__, x =colnames(data__)[2] ,y = colnames(data__)[i],
          add = "reg.line", conf.int = T,
          color = "risk",
          add.params = list(fill = "lightgray",color = "black"),
          ggtheme = theme_minimal(), margin.plot =  "density"
)+ stat_cor(method = "pearson",
           label.x = median(data__[,2]), label.y =max(data__[,i]),color='red',size =7)+theme_global+
    scale_color_manual(values =risk_colors ,guide = F)
g2 = ggMarginal(g1, type="density",xparams = list( fill="#3c69ac", size = 0.5,alpha=1), yparams = list(fill="#bb4962", size = 0.5,alpha=1))
pp[[i]] = g2
}
cor_4_hallmark = data__
save(cor_4_hallmark,file = "cor_4_hallmark.rdata")
plot_ = pp[4:7]
pdf("cor_list.pdf",width = 20,height = 5)
cowplot::plot_grid(plotlist = plot_, align = "h", nrow = 1)
dev.off()